In [18]:
import cv2
from scripts import basic_image_operations as basic_ops
import numpy as np

%store -r stop_templates
%store -r vf_templates
%store -r vfa_templates
%store -r vfs_templates

%store -r resized_images
%store -r shape_images

In [19]:
import cv2
import numpy as np
import copy

template_lists = [stop_templates, vf_templates, vfa_templates, vfs_templates]

resized_images_copy = copy.deepcopy(resized_images)

def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)


def edge_based_matching(image, template_lists, threshold=0.4):
    image_edges = image
    results = []

    for i, template_list in enumerate(template_lists):
        best_score = -np.inf
        best_coords = None
        best_shape = None

        for template in template_list:
            template_edges = template

            if image_edges.shape[0] < template_edges.shape[0] or image_edges.shape[1] < template_edges.shape[1]:
                continue

            heatmap = cv2.matchTemplate(image_edges, template_edges, cv2.TM_CCOEFF_NORMED)
            score = np.max(heatmap)
            coords = np.unravel_index(np.argmax(heatmap), heatmap.shape)

            if score > best_score:
                best_score = score
                best_coords = coords
                best_shape = template_edges.shape

        results.append((best_score, best_coords, best_shape))

    return results

# Run edge-based matching on all shape_images
for img_idx, image in enumerate(shape_images):
    print(f'\nProcessing Image {img_idx}:')
    results = edge_based_matching(image, template_lists, threshold=0.4)

    for list_idx, (score, coords, shape) in enumerate(results):
        if score >= 0.4:
            print(f'  Template List {list_idx}: Best match at (row, col) = {coords}, score = {score:.3f}')
            top_left = (coords[1], coords[0])
            bottom_right = (top_left[0] + shape[1], top_left[1] + shape[0])
            cv2.rectangle(resized_images_copy[img_idx], top_left, bottom_right, (0, 255, 0), 2)
        else:
            print(f'  Template List {list_idx}: No good match (score = {score:.3f})')

    cv2.imshow(f"Image {img_idx} Matches", resized_images_copy[img_idx])
    cv2.waitKey(0)
    cv2.destroyAllWindows()



Processing Image 0:
  Template List 0: No good match (score = 0.267)
  Template List 1: No good match (score = 0.326)
  Template List 2: No good match (score = 0.300)
  Template List 3: No good match (score = 0.244)

Processing Image 1:
  Template List 0: No good match (score = 0.279)
  Template List 1: No good match (score = 0.359)
  Template List 2: No good match (score = 0.245)
  Template List 3: No good match (score = 0.240)

Processing Image 2:
  Template List 0: No good match (score = 0.307)
  Template List 1: No good match (score = 0.328)
  Template List 2: No good match (score = 0.374)
  Template List 3: No good match (score = 0.319)

Processing Image 3:
  Template List 0: No good match (score = 0.256)
  Template List 1: Best match at (row, col) = (np.int64(22), np.int64(114)), score = 0.408
  Template List 2: No good match (score = 0.313)
  Template List 3: No good match (score = 0.195)

Processing Image 4:
  Template List 0: No good match (score = 0.296)
  Template List 1: N

In [8]:
for img_idx, image in enumerate(shape_images):
    print(f'\nProcessing Image {img_idx}:')
    for i, template_list in enumerate(template_lists):
        print(f'  Template List {i}:')
        best_score = -np.inf
        best_coords = None
        for template in template_list:
            if image.shape[0] < template.shape[0] or image.shape[1] < template.shape[1]:
                continue

            heatmap = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
            score = np.max(heatmap)
            coords = np.unravel_index(np.argmax(heatmap), heatmap.shape)

            if score > best_score:
                best_score = score
                best_coords = coords

        if best_coords is not None:
            print(f'        Best match at: (row, col) = {best_coords}, score = {best_score:.3f}')



Processing Image 0:
  Template List 0:
        Best match at: (row, col) = (np.int64(199), np.int64(337)), score = 0.267
  Template List 1:
        Best match at: (row, col) = (np.int64(400), np.int64(166)), score = 0.326
  Template List 2:
        Best match at: (row, col) = (np.int64(382), np.int64(188)), score = 0.300
  Template List 3:
        Best match at: (row, col) = (np.int64(156), np.int64(89)), score = 0.244

Processing Image 1:
  Template List 0:
        Best match at: (row, col) = (np.int64(129), np.int64(25)), score = 0.279
  Template List 1:
        Best match at: (row, col) = (np.int64(258), np.int64(105)), score = 0.359
  Template List 2:
        Best match at: (row, col) = (np.int64(116), np.int64(224)), score = 0.245
  Template List 3:
        Best match at: (row, col) = (np.int64(97), np.int64(225)), score = 0.240

Processing Image 2:
  Template List 0:
        Best match at: (row, col) = (np.int64(46), np.int64(23)), score = 0.307
  Template List 1:
        Best m

In [9]:
for image in shape_images:
    basic_ops.show_image(image)

Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
